# 지하철 1호선 배차 간격 분석
## Headway Analysis - 열차가 얼마나 자주 오는가?

### 배차 간격(Headway)이란?
- 같은 역에서 연속된 두 열차가 도착하는 시간 간격
- 승객이 열차를 얼마나 기다려야 하는지를 나타내는 핵심 지표

### 분석 목적
1. 역별 평균 배차 간격 파악
2. 배차 간격이 긴 역 식별 (승객 대기 시간 증가)
3. 시간대별 배차 패턴 분석
4. 주중/주말 운행 빈도 비교
5. 배차 간격의 일관성 평가 (변동성)

In [10]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import sqlite3
import numpy as np
import warnings
warnings.filterwarnings('ignore')

DB_PATH = r"..\data\input\metro_datetime.db"
TABLE = 'Metro_Line_1_Schedule'

## 1. 데이터 로드

In [11]:
# 데이터 로드
with sqlite3.connect(DB_PATH) as conn:
    query = f'''
        SELECT 
            역사코드, 역사명, 주중주말, 방향, 급행여부,
            열차코드, 열차도착시간_dt, 출발역, 도착역
        FROM {TABLE}
        WHERE 급행여부 = 0
        ORDER BY 역사코드, 방향, 열차도착시간_dt
    '''
    df = pd.read_sql(query, conn)

# datetime 변환
df['열차도착시간_dt'] = pd.to_datetime(df['열차도착시간_dt'])
df['시간대'] = df['열차도착시간_dt'].dt.hour

print(f'📊 전체 데이터: {len(df):,}행')
print(f'🚉 고유 역 수: {df["역사명"].nunique():,}개')
print(f'🚇 고유 열차 수: {df["열차코드"].nunique():,}대')
display(df.head())

📊 전체 데이터: 68,791행
🚉 고유 역 수: 102개
🚇 고유 열차 수: 708대


,역사코드,역사명,주중주말,방향,급행여부,열차코드,열차도착시간_dt,출발역,도착역,시간대
0,150,서울역,DAY,DOWN,0,S401,NaT,서울역,서동탄,NaN
1,150,서울역,END,DOWN,0,S401,NaT,서울역,서동탄,NaN
2,150,서울역,SAT,DOWN,0,S401,NaT,서울역,서동탄,NaN
3,150,서울역,DAY,DOWN,0,K11,2025-07-04 05:30:00,광운대,인천,5.0
4,150,서울역,END,DOWN,0,K7,2025-07-04 05:30:00,광운대,인천,5.0


## 2. 배차 간격 계산

In [12]:
def calculate_headway(df):
    '''역별, 방향별, 주중주말별 배차 간격 계산'''
    records = []
    
    for (station, direction, day_type), group in df.groupby(['역사명', '방향', '주중주말']):
        group = group[group['열차도착시간_dt'].notna()].copy()
        group = group.sort_values('열차도착시간_dt').reset_index(drop=True)
        
        if len(group) > 1:
            for i in range(1, len(group)):
                prev = group.iloc[i-1]
                curr = group.iloc[i]
                
                headway = (curr['열차도착시간_dt'] - prev['열차도착시간_dt']).total_seconds() / 60
                
                if 1 <= headway <= 120:
                    records.append({
                        '역사명': station,
                        '역사코드': curr['역사코드'],
                        '방향': direction,
                        '주중주말': day_type,
                        '배차간격_분': headway,
                        '시간대': curr['시간대']
                    })
    
    return pd.DataFrame(records)

print('⏱️ 배차 간격 계산 중...')
df_headway = calculate_headway(df)

print(f'\✅ 총 {len(df_headway):,}개의 배차 간격 기록')
print(f'평균: {df_headway["배차간격_분"].mean():.2f}분')
print(f'중앙값: {df_headway["배차간격_분"].median():.2f}분')
display(df_headway.head(20))

⏱️ 배차 간격 계산 중...
\✅ 총 66,483개의 배차 간격 기록
평균: 10.08분
중앙값: 8.50분


,역사명,역사코드,방향,주중주말,배차간격_분,시간대
0,가능,1907,DOWN,DAY,9.0,5.0
1,가능,1907,DOWN,DAY,24.0,5.0
2,가능,1907,DOWN,DAY,18.0,6.0
3,가능,1907,DOWN,DAY,14.5,6.0
4,가능,1907,DOWN,DAY,19.0,6.0
5,가능,1907,DOWN,DAY,13.5,6.0
6,가능,1907,DOWN,DAY,7.5,6.0
7,가능,1907,DOWN,DAY,14.0,7.0
8,가능,1907,DOWN,DAY,13.5,7.0
9,가능,1907,DOWN,DAY,6.5,7.0


## 3. 역별 평균 배차 간격 통계

In [13]:
# 역별 통계
station_stats = df_headway.groupby(['역사명', '역사코드', '방향', '주중주말']).agg({
    '배차간격_분': ['mean', 'median', 'std', 'min', 'max', 'count']
}).round(2)

station_stats.columns = ['평균_분', '중앙값_분', '표준편차', '최소_분', '최대_분', '측정횟수']
station_stats = station_stats.reset_index()
station_sorted = station_stats.sort_values('평균_분', ascending=False)

print('⏱️ 평균 배차 간격이 긴 역 Top 30:')
display(station_sorted.head(30))

# CSV 저장
station_sorted.to_csv('..\data\output\역별_배차간격_전체.csv', index=False, encoding='utf-8-sig')
print('\✅ 역별_배차간격_전체.csv 저장 완료')

⏱️ 평균 배차 간격이 긴 역 Top 30:


,역사명,역사코드,방향,주중주말,평균_분,중앙값_분,표준편차,최소_분,최대_분,측정횟수
557,청산,1917,DOWN,SAT,60.17,61.0,4.15,53.0,67.0,18
556,청산,1917,DOWN,END,60.17,61.0,4.15,53.0,67.0,18
472,전곡,1918,DOWN,END,60.17,61.0,4.15,53.0,67.0,18
473,전곡,1918,DOWN,SAT,60.17,61.0,4.15,53.0,67.0,18
313,소요산,1916,DOWN,END,60.17,61.0,4.15,53.0,67.0,18
314,소요산,1916,DOWN,SAT,60.17,61.0,4.15,53.0,67.0,18
475,전곡,1918,UP,END,60.00,58.5,5.60,53.0,71.5,18
476,전곡,1918,UP,SAT,60.00,58.5,5.60,53.0,71.5,18
400,연천,1919,UP,END,60.00,58.5,5.60,53.0,71.5,18
401,연천,1919,UP,SAT,60.00,58.5,5.60,53.0,71.5,18


\✅ 역별_배차간격_전체.csv 저장 완료


## 4. 주중 배차 간격 Top 30

In [14]:
# 주중 데이터
weekday = station_stats[station_stats['주중주말'] == 'DAY'].copy()
weekday_sorted = weekday.sort_values('평균_분', ascending=False)

print('⚠️ 주중 평균 배차 간격이 긴 역 Top 30:')
display(weekday_sorted.head(30))

weekday_sorted.to_csv('..\data\output\역별_배차간격_Top30_주중.csv', index=False, encoding='utf-8-sig')
print('✅ 역별_배차간격_Top30_주중.csv 저장 완료')

⚠️ 주중 평균 배차 간격이 긴 역 Top 30:


,역사명,역사코드,방향,주중주말,평균_분,중앙값_분,표준편차,최소_분,최대_분,측정횟수
558,청산,1917,UP,DAY,54.85,59.00,13.71,28.0,73.0,20
474,전곡,1918,UP,DAY,54.85,59.00,13.77,24.0,73.0,20
399,연천,1919,UP,DAY,54.85,59.00,12.48,27.5,70.0,20
312,소요산,1916,DOWN,DAY,54.05,56.00,10.74,30.0,67.0,20
471,전곡,1918,DOWN,DAY,54.05,56.00,10.74,30.0,67.0,20
555,청산,1917,DOWN,DAY,54.05,56.00,10.74,30.0,67.0,20
30,광명,1750,DOWN,DAY,53.68,51.00,11.20,39.5,76.0,19
378,아산,1403,UP,DAY,35.16,30.00,16.87,11.0,77.0,32
234,봉명,1401,UP,DAY,35.16,30.00,16.87,11.0,77.0,32
210,배방,1405,UP,DAY,35.16,30.00,16.87,11.0,77.0,32


✅ 역별_배차간격_Top30_주중.csv 저장 완료


## 5. 주말 배차 간격 Top 30

In [15]:
# 주말 데이터
weekend = station_stats[station_stats['주중주말'] == 'SAT'].copy()
weekend_sorted = weekend.sort_values('평균_분', ascending=False)

print('⚠️ 주말 평균 배차 간격이 긴 역 Top 30:')
display(weekend_sorted.head(30))

weekend_sorted.to_csv('..\data\output\역별_배차간격_Top30_주말.csv', index=False, encoding='utf-8-sig')
print('✅ 역별_배차간격_Top30_주말.csv 저장 완료')

⚠️ 주말 평균 배차 간격이 긴 역 Top 30:


,역사명,역사코드,방향,주중주말,평균_분,중앙값_분,표준편차,최소_분,최대_분,측정횟수
557,청산,1917,DOWN,SAT,60.17,61.00,4.15,53.0,67.0,18
473,전곡,1918,DOWN,SAT,60.17,61.00,4.15,53.0,67.0,18
314,소요산,1916,DOWN,SAT,60.17,61.00,4.15,53.0,67.0,18
401,연천,1919,UP,SAT,60.00,58.50,5.60,53.0,71.5,18
476,전곡,1918,UP,SAT,60.00,58.50,5.60,53.0,71.5,18
560,청산,1917,UP,SAT,60.00,58.50,5.60,53.0,71.5,18
32,광명,1750,DOWN,SAT,53.68,51.00,11.20,39.5,76.0,19
368,신창,1408,DOWN,SAT,36.47,33.00,18.08,14.0,85.0,29
233,봉명,1401,DOWN,SAT,36.47,33.00,18.08,14.0,85.0,29
209,배방,1405,DOWN,SAT,36.47,33.00,18.08,14.0,85.0,29


✅ 역별_배차간격_Top30_주말.csv 저장 완료


## 6. 주중/주말 비교

In [16]:
# 주중/주말 비교
comparison = station_stats.pivot_table(
    index=['역사명', '방향'],
    columns='주중주말',
    values='평균_분'
).round(2).reset_index()

if 'DAY' in comparison.columns and 'SAT' in comparison.columns:
    comparison['차이(주중-주말)'] = comparison['DAY'] - comparison['SAT']
    comparison['평균'] = comparison[['DAY', 'SAT']].mean(axis=1).round(2)
    comparison_sorted = comparison.sort_values('평균', ascending=False)
    
    print('📊 주중/주말 배차 간격 비교 Top 30:')
    display(comparison_sorted.head(30))
    
    comparison_sorted.to_csv('..\data\output\배차간격_주중주말비교.csv', index=False, encoding='utf-8-sig')
    print('\✅ 배차간격_주중주말비교.csv 저장 완료')

📊 주중/주말 배차 간격 비교 Top 30:


주중주말,역사명,방향,DAY,END,SAT,차이(주중-주말),평균
186,청산,UP,54.85,60.00,60.00,-5.15,57.42
158,전곡,UP,54.85,60.00,60.00,-5.15,57.42
133,연천,UP,54.85,60.00,60.00,-5.15,57.42
104,소요산,DOWN,54.05,60.17,60.17,-6.12,57.11
157,전곡,DOWN,54.05,60.17,60.17,-6.12,57.11
185,청산,DOWN,54.05,60.17,60.17,-6.12,57.11
10,광명,DOWN,53.68,53.68,53.68,0.00,53.68
122,신창,DOWN,34.81,36.47,36.47,-1.66,35.64
77,봉명,DOWN,34.81,36.47,36.47,-1.66,35.64
69,배방,DOWN,34.81,36.47,36.47,-1.66,35.64


\✅ 배차간격_주중주말비교.csv 저장 완료


## 7. 변동성이 큰 역 (불안정한 역)

In [17]:
# 표준편차가 큰 역
variance = weekday.copy()
variance_sorted = variance.sort_values('표준편차', ascending=False)

print('⚠️ 배차 간격 변동성이 큰 역 Top 30 (주중):')
display(variance_sorted.head(30))

variance_sorted.to_csv('..\data\output\배차간격_변동성_높은역.csv', index=False, encoding='utf-8-sig')
print('✅ 배차간격_변동성_높은역.csv 저장 완료')

⚠️ 배차 간격 변동성이 큰 역 Top 30 (주중):


,역사명,역사코드,방향,주중주말,평균_분,중앙값_분,표준편차,최소_분,최대_분,측정횟수
207,배방,1405,DOWN,DAY,34.81,29.25,19.18,13.0,96.5,32
432,온양온천,1407,DOWN,DAY,34.81,29.25,19.18,13.0,96.5,32
561,탕정,1404,DOWN,DAY,34.81,29.25,19.18,13.0,96.5,32
369,쌍용,1402,DOWN,DAY,34.81,29.25,19.18,13.0,96.5,32
375,아산,1403,DOWN,DAY,34.81,29.25,19.18,13.0,96.5,32
366,신창,1408,DOWN,DAY,34.81,29.25,19.18,13.0,96.5,32
231,봉명,1401,DOWN,DAY,34.81,29.25,19.18,13.0,96.5,32
234,봉명,1401,UP,DAY,35.16,30.00,16.87,11.0,77.0,32
372,쌍용,1402,UP,DAY,35.16,30.00,16.87,11.0,77.0,32
210,배방,1405,UP,DAY,35.16,30.00,16.87,11.0,77.0,32


✅ 배차간격_변동성_높은역.csv 저장 완료


## 8. 배차 간격이 짧은 역 (서비스 좋은 역)

In [18]:
# 배차 간격이 짧은 역
short = weekday.copy()
short_sorted = short.sort_values('평균_분')

print('✅ 배차 간격이 짧은 역 Top 30 (주중):')
display(short_sorted.head(30))

short_sorted.to_csv('..\data\output\배차간격_짧은역_Top30.csv', index=False, encoding='utf-8-sig')
print('✅ 배차간격_짧은역_Top30.csv 저장 완료')

✅ 배차 간격이 짧은 역 Top 30 (주중):


,역사명,역사코드,방향,주중주말,평균_분,중앙값_분,표준편차,최소_분,최대_분,측정횟수
42,구로,1701,UP,DAY,4.86,4.0,2.38,1.0,19.5,235
351,신도림,1007,UP,DAY,5.07,4.5,2.23,1.0,12.5,225
405,영등포,1006,UP,DAY,5.07,4.5,2.23,1.0,12.5,225
348,신도림,1007,DOWN,DAY,5.26,4.5,2.56,1.0,21.0,221
39,구로,1701,DOWN,DAY,5.26,4.5,2.56,1.0,21.0,221
150,동대문,155,UP,DAY,5.43,4.5,2.60,2.5,26.0,215
168,동묘앞,159,UP,DAY,5.43,4.5,2.60,2.5,26.0,215
339,시청,151,UP,DAY,5.43,4.5,2.60,2.5,26.0,215
498,종로3가,153,UP,DAY,5.43,4.5,2.60,2.5,26.0,215
492,종각,152,UP,DAY,5.43,4.5,2.60,2.5,26.0,215


✅ 배차간격_짧은역_Top30.csv 저장 완료


## 9. 시각화: 배차 간격 Top 30

In [19]:
# Top 30 시각화
top30 = weekday_sorted.head(30).copy()
top30['역-방향'] = top30['역사명'] + ' (' + top30['방향'] + ')'

fig = px.bar(
    top30,
    x='평균_분',
    y='역-방향',
    orientation='h',
    title='평균 배차 간격 Top 30 (주중)',
    labels={'평균_분': '평균 배차 간격 (분)', '역-방향': '역 (방향)'},
    color='평균_분',
    color_continuous_scale='Reds'
)

fig.update_layout(height=900, yaxis={'categoryorder': 'total ascending'})
fig.show()

## 10. 시각화: 배차 간격 분포

In [20]:
# 주중/주말 분포 비교
fig = px.histogram(
    df_headway,
    x='배차간격_분',
    color='주중주말',
    nbins=60,
    title='배차 간격 분포 (주중 vs 주말)',
    barmode='overlay',
    opacity=0.7
)
fig.show()

print('\📊 배차 간격 통계:')
display(df_headway.groupby('주중주말')['배차간격_분'].describe())

\📊 배차 간격 통계:


,count,mean,std,min,25%,50%,75%,max
주중주말,,,,,,,,
DAY,22527.0,9.976118,7.946655,1.0,5.0,8.0,11.5,96.5
END,21978.0,10.130244,7.847302,1.0,5.0,8.5,12.0,85.0
SAT,21978.0,10.130244,7.847302,1.0,5.0,8.5,12.0,85.0


## 11. 시각화: 평균 vs 변동성

In [21]:
# 산점도
weekday_viz = weekday.copy()
weekday_viz['역-방향'] = weekday_viz['역사명'] + ' (' + weekday_viz['방향'] + ')'

fig = px.scatter(
    weekday_viz,
    x='평균_분',
    y='표준편차',
    size='측정횟수',
    color='평균_분',
    hover_data=['역-방향'],
    title='배차 간격: 평균 vs 변동성 (주중)',
    color_continuous_scale='Reds'
)
fig.show()

## 12. 시간대별 배차 간격

In [22]:
# 시간대별 평균
hourly = df_headway[df_headway['주중주말'] == 'DAY'].groupby('시간대').agg({
    '배차간격_분': ['mean', 'count']
}).round(2)
hourly.columns = ['평균_분', '측정횟수']
hourly = hourly.reset_index()

print('⏰ 시간대별 평균 배차 간격 (주중):')
display(hourly)

# 시각화
fig = px.line(
    hourly,
    x='시간대',
    y='평균_분',
    title='시간대별 평균 배차 간격 (주중)',
    markers=True
)
fig.show()

⏰ 시간대별 평균 배차 간격 (주중):


,시간대,평균_분,측정횟수
0,0.0,15.45,303
1,1.0,28.25,2
2,5.0,10.71,517
3,6.0,9.40,1291
4,7.0,7.68,1507
5,8.0,8.37,1451
6,9.0,9.59,1219
7,10.0,10.86,1111
8,11.0,11.08,1058
9,12.0,12.10,989


## 13. 개선 우선순위

In [23]:
# 우선순위 점수
priority = weekday.copy()

priority['배차점수'] = ((priority['평균_분'] - priority['평균_분'].min()) / 
                      (priority['평균_분'].max() - priority['평균_분'].min()) * 100).round(2)

priority['변동점수'] = ((priority['표준편차'] - priority['표준편차'].min()) / 
                      (priority['표준편차'].max() - priority['표준편차'].min()) * 100).round(2)

priority['종합점수'] = (priority['배차점수'] * 0.7 + priority['변동점수'] * 0.3).round(2)

def recommend(row):
    if row['평균_분'] > 15:
        return '증편+정시성' if row['표준편차'] > 5 else '증편 우선'
    elif row['표준편차'] > 5:
        return '정시성 우선'
    return '현상 유지'

priority['개선방안'] = priority.apply(recommend, axis=1)

priority_list = priority.nlargest(30, '종합점수')[[
    '역사명', '방향', '평균_분', '표준편차', '배차점수', '변동점수', '종합점수', '개선방안'
]].reset_index(drop=True)

print('🎯 개선 우선순위 Top 30:')
display(priority_list)

priority_list.to_csv('..\data\output\배차간격_개선우선순위_Top30.csv', index=False, encoding='utf-8-sig')
print('\✅ 배차간격_개선우선순위_Top30.csv 저장 완료')

🎯 개선 우선순위 Top 30:


,역사명,방향,평균_분,표준편차,배차점수,변동점수,종합점수,개선방안
0,전곡,UP,54.85,13.77,100.00,68.08,90.42,증편+정시성
1,청산,UP,54.85,13.71,100.00,67.73,90.32,증편+정시성
2,연천,UP,54.85,12.48,100.00,60.47,88.14,증편+정시성
3,광명,DOWN,53.68,11.20,97.66,52.92,84.24,증편+정시성
4,소요산,DOWN,54.05,10.74,98.40,50.21,83.94,증편+정시성
5,전곡,DOWN,54.05,10.74,98.40,50.21,83.94,증편+정시성
6,청산,DOWN,54.05,10.74,98.40,50.21,83.94,증편+정시성
7,배방,DOWN,34.81,19.18,59.91,100.00,71.94,증편+정시성
8,봉명,DOWN,34.81,19.18,59.91,100.00,71.94,증편+정시성
9,신창,DOWN,34.81,19.18,59.91,100.00,71.94,증편+정시성


\✅ 배차간격_개선우선순위_Top30.csv 저장 완료
